In [2]:
# we first load the class module

import dipolarBEC

In [3]:
# necessary python modules

from tqdm import tqdm
import numpy as np
import os
import pandas as pd
from scipy.optimize import curve_fit
from scipy.special import kn
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl        
mpl.rcParams['text.usetex'] = True
import seaborn
font = {'family' : 'Times New Roman',
        'weight' : 'bold',
        'size'   : 14}
mpl.rc('font', **font)

pal = seaborn.color_palette("tab10")
print(pal.as_hex())

# if seaborn does not work, try: pip install seaborn

['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']


In [4]:
#check which interaction is used and save fig in the appropriate subfolder

fv = 'd'  # 'd' or 'NN'

path_dict = {
    'd': "C:\\Users\\camipolv\\Desktop\\dipbec\\fig\\dip\\",
    'NN': "C:\\Users\\camipolv\\Desktop\\dipbec\\fig\\NN\\"
}

fpath = path_dict.get(fv, "Default path")

Ust = r"$U_{NN}$" if fv == "NN" else r"$U_d$" if fv == "d" else None
Ucs = r"$/U_c$"

In [5]:
#parameters

Ndisr = 100
Nmesh = 1000

N0 = 2
N1=100
N2=250
N3=500

kx_small = 0.01
kx_large_dict = {'d': 8.0, 'NN': 25.0}
kx_large = kx_large_dict.get(fv, "Default value")

Uc = 1.0

#NN: Ud << Uc/2, d: Ud << Uc/3
Ud1_dict = {'d': 0.0, 'NN': 0.0}
Ud2_dict = {'d': 0.2, 'NN': 0.2}
Ud3_dict = {'d': 0.3, 'NN': 0.5}
Ud1 = Ud1_dict.get(fv, "Default value")
Ud2 = Ud2_dict.get(fv, "Default value")
Ud3 = Ud3_dict.get(fv, "Default value")

sigma1 = 0.0
sigma2 = 0.2
sigma3 = 0.4

t1 = 1.0
t2 = 5.0
t3 = 7.0

Ns = [N1, N2, N3]
kxs = [kx_small, kx_large]
sigmas = [sigma1, sigma2, sigma3]
Uds = [Ud1, Ud2, Ud3]

markers = ['s', 'o', '^']

In [6]:
#riemann sum integration - used for the viscosity
def intg(kxar,far):
    intg = (kxar[-1]-kxar[0])*np.sum(far)/len(kxar)
    return intg

In [ ]:
# plot of viscosity integrand (no oscillatory part) vs kx for fix sigma, Ntubes=2 (ny=1), and 3 values of Ud

Ntubes = N0
ny = Ntubes-1
sigma = sigmas[1]

kxar = np.linspace(kxs[0], kxs[1], Nmesh)

visc_kx = [[] for _ in range(len(Uds))]

nb = np.random.uniform(1-sigma, 1+sigma, Ntubes)

for kx in tqdm(kxar):
    for i, ud in enumerate(Uds):
        run_k = dipolarBEC.dipolarBEC(Ntubes, kx, Uc, ud, Ndisr, sigma)
        visc_kx[i].append(np.abs(run_k.visc_k_ij(ny,1,2,nb)))

In [ ]:
plt.figure(figsize=(5,4))

for i in range(len(Uds)):
    plt.plot(kxar, visc_kx[i], label=Ust+Ucs+r'= {}'.format(Uds[i]), marker=markers[i])

plt.xlabel(r"$k_x$", fontsize=14)
plt.ylabel(r"Shear Viscosity Kernel", fontsize=14)
plt.legend(loc='best', fontsize=14)
#plt.title(r"$N = {}$ tubes, $n = {}$ tube distance, $\sigma = {}$".format(Ntubes,ny, sigma), fontsize=14)
plt.savefig(os.path.join(fpath, r'visc_intd_vs_k_N=2_sigma={}.jpg'.format(sigma)), format='jpg', bbox_inches='tight')
plt.savefig(os.path.join(fpath, r'visc_intd_vs_k_N=2_sigma={}.pdf'.format(sigma)), format='pdf', bbox_inches='tight')

plt.show()

In [ ]:
# viscosity vs Ud/Uc for a fixed Ntubes, sigma, ny and t

Ntubes = N0
ny = Ntubes-1

t = t1

Udar = np.linspace(Ud1, Ud3, 10)
kxar = np.linspace(kxs[0], kxs[1], Nmesh)

nbs = [np.random.uniform(1-s, 1+s, Ntubes) for s in sigmas]
visc_ud = [[], [], []]

for ud in tqdm(Udar):
    visc_kx = [[], [], []]
    for kx in tqdm(kxar):
        for i in range(len(sigmas)):
            run_k = dipolarBEC.dipolarBEC(Ntubes, kx, Uc, ud, Ndisr, sigmas[i])
            visc_kx[i].append(run_k.visc_k_time(ny, t, nbs[i]))
    for i in range(len(visc_ud)):
        visc_ud[i].append(intg(kxar, visc_kx[i]))

In [ ]:
plt.figure(figsize=(5,4))

for i in range(len(visc_ud)):
    plt.plot(Udar, visc_ud[i], label=r'$\sigma = {}$'.format(sigmas[i]), marker=markers[i])

plt.xlabel(Ust+Ucs, fontsize=14)
plt.ylabel(r"Shear Viscosity at fixed t", fontsize=14)
plt.legend(loc='best', fontsize=14)
#plt.title(r"$N = {}$ tubes, $n = {}$ tube distance, t = {}".format(Ntubes,ny,t), fontsize=14)
plt.savefig(os.path.join(fpath, r'visc_vs_ud_t={}.jpg'.format(t)), format='jpg', bbox_inches='tight')
plt.savefig(os.path.join(fpath, r'visc_vs_ud_t={}.pdf'.format(t)), format='pdf', bbox_inches='tight')

plt.show()

In [ ]:
# now do visc as a function of sigma for three different Ud/Uc

In [ ]:
#plot

In [8]:
# viscosity vs t for 3 Ud/Uc, fixed Ntubes, sigma, ny

Ntar = [2,5,10]
nys = [n-1 for n in Ntar]
sigma = sigmas[1]
Ud = Uds[1]

tdar = np.arange(0, t2, .01)
kxar = np.linspace(kxs[0], kxs[1], Nmesh)

nbs = [np.random.uniform(1-sigma, 1+sigma, n) for n in Ntar]
visc_t = [[], [], []]

## Create a list to store the dipolarBEC objects
#runs = [[], [], []]

# Create the dipolarBEC objects before the t loop
#for kx in tqdm(kxar):
#   for i, ud in enumerate(Uds):
        #run_k = dipolarBEC.dipolarBEC(Ntubes, kx, Uc, ud, Ndisr, sigma)
        #runs[i].append(run_k)


for t in tqdm(tdar):
    visc0_kx = [[], [], []]
    visc_kx = [[], [], []]
    for kx in tqdm(kxar):
        for i, n in enumerate(Ntar):
            run_k = dipolarBEC.dipolarBEC(n, kx, Uc, Ud, Ndisr, sigma)
            visc0_kx[i].append(run_k.visc_k_0(nys[i], nbs[i]))
            visc_kx[i].append(run_k.visc_k_time(nys[i], t, nbs[i]))
    for i in range(len(visc_t)):
        visc_t[i].append((intg(kxar, visc_kx[i]))/(intg(kxar, visc0_kx[2])))

  2%|▏         | 12/500 [1:35:45<64:54:22, 478.82s/it]


KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(5,4))

for i in range(len(visc_t)):
    plt.plot(tdar, visc_t[i], label=Ust+Ucs+r'= {}'.format(Ntar[i]))

plt.xlabel(Ust+r"$\,t$", fontsize=14)
plt.ylabel(r"$\eta/\eta_0$", fontsize=14)
plt.legend(loc='best', fontsize=14)
#plt.title(r"$N = {}$ tubes, $n = {}$ tube distance, $\sigma = {}$".format(Ntubes,ny, sigma), fontsize=14)
#plt.savefig(os.path.join(fpath, r'visc_vs_t_3ud.jpg'), format='jpg', bbox_inches='tight')
#plt.savefig(os.path.join(fpath, r'visc_vs_t_3ud.pdf'), format='pdf', bbox_inches='tight')


plt.show()

In [7]:
# viscosity vs t for 3 Ud/Uc, fixed Ntubes, sigma, ny - dipbec before t-loop

Nmesh=100
Ntubes = N0
ny = Ntubes-1
sigma = sigmas[1]

tdar = np.arange(0, t2, .01)
kxar = np.linspace(kxs[0], kxs[1], Nmesh)

nb = np.random.uniform(1-sigma, 1+sigma, Ntubes)
visc_t = [[], [], []]

# Create a list to store the dipolarBEC objects
runs = [[], [], []]

# Create the dipolarBEC objects before the t loop
for kx in tqdm(kxar):
    for i, ud in enumerate(Uds):
        run_k = dipolarBEC.dipolarBEC(Ntubes, kx, Uc, ud, Ndisr, sigma)
        runs[i].append(run_k)

for t in tqdm(tdar):
    visc0_kx = [[], [], []]
    visc_kx = [[], [], []]
    for kx_index, kx in enumerate(tqdm(kxar)):
        for i, ud in enumerate(Uds):
            # Use the previously created dipolarBEC object
            run_k = runs[i][kx_index]
            visc0_kx[i].append(run_k.visc_k_0(ny, nb))
            visc_kx[i].append(run_k.visc_k_time(ny, t, nb))
    for i in range(len(visc_t)):
        visc_t[i].append((intg(kxar, visc_kx[i]))/(intg(kxar, visc0_kx[2])))

  0%|          | 0/500 [00:00<?, ?it/s]

100%|██████████| 500/500 [06:37<00:00,  1.26it/s]


In [1]:
plt.figure(figsize=(5,4))

for i in range(len(visc_t)):
    plt.plot(tdar, visc_t[i], label=Ust+Ucs+r'= {}'.format(Uds[i]))

plt.xlabel(Ust+r"$\,t$", fontsize=14)
plt.ylabel(r"$\eta/\eta_0$", fontsize=14)
plt.legend(loc='best', fontsize=14)
#plt.title(r"$N = {}$ tubes, $n = {}$ tube distance, $\sigma = {}$".format(Ntubes,ny, sigma), fontsize=14)
plt.savefig(os.path.join(fpath, r'visc_vs_t_3ud.jpg'), format='jpg', bbox_inches='tight')
plt.savefig(os.path.join(fpath, r'visc_vs_t_3ud.pdf'), format='pdf', bbox_inches='tight')


plt.show()

NameError: name 'plt' is not defined

In [ ]:
# viscosity vs t for 3 sigma, and fixed Ntubes,  Ud/Uc, ny

Ntubes = N0
ny = Ntubes-1
Ud = Uds[2]

tdar = np.arange(0, t2, .01)
kxar = np.linspace(kx_small, kx_large, Nmesh)

nbs = [np.random.uniform(1-s, 1+s, Ntubes) for s in sigmas]
print(f'nb = {nbs}')
visc_t = [[], [], []]

for t in tqdm(tdar):
    visc0_kx = [[], [], []]
    visc_kx = [[], [], []]
    for kx in tqdm(kxar):
        for i, s in enumerate(sigmas):
            run_k = dipolarBEC.dipolarBEC(Ntubes, kx, Uc, Ud, Ndisr, s)
            visc0_kx[i].append(run_k.visc_k_0(ny, nbs[i]))
            visc_kx[i].append(run_k.visc_k_time(ny, t, nbs[i]))
    for i in range(len(visc_t)):
        visc_t[i].append((intg(kxar, visc_kx[i]))/(intg(kxar, visc0_kx[2])))

nb = [array([1., 1.]), array([0.88884739, 1.09352311]), array([0.70009153, 0.75716992])]


100%|██████████| 500/500 [50:34<00:00,  6.07s/it]


In [ ]:
#diagonalization outside the t loop
Ntubes = N0
ny = Ntubes-1
Ud = Uds[2]

tdar = np.arange(0, t2, .01)
kxar = np.linspace(kx_small, kx_large, Nmesh)

nbs = [np.random.uniform(1-s, 1+s, Ntubes) for s in sigmas]
print(f'nb = {nbs}')
visc_t = [[], [], []]

# Create a list to store the dipolarBEC objects
runs = [[], [], []]

# Create the dipolarBEC objects before the t loop
for kx in tqdm(kxar):
    for i, s in enumerate(sigmas):
        run_k = dipolarBEC.dipolarBEC(Ntubes, kx, Uc, Ud, Ndisr, s)
        runs[i].append(run_k)

# Compute visc0_kx before entering the t loop
visc0_kx = [[run_k.visc_k_0(ny, nbs[i]) for run_k in run_list] for i, run_list in enumerate(runs)]

for t in tqdm(tdar):
    visc_kx = [[], [], []]
    visc0_kx = [[],[],[]]
    for kx_index, kx in enumerate(tqdm(kxar)):
        for i, s in enumerate(sigmas):
            # Use the previously created dipolarBEC object
            run_k = runs[i][kx_index]
            visc0_kx[i].append(run_k.visc_k_0(ny, nbs[i]))
            visc_kx[i].append(run_k.visc_k_time(ny, t, nbs[i]))
    for i in range(len(visc_t)):
        visc_t[i].append(intg(kxar, visc_kx[i])/intg(kxar, visc0_kx[2]))

In [ ]:

for kx in tqdm(kxar):
    for i, s in enumerate(sigmas):
        run_k = dipolarBEC.dipolarBEC(Ntubes, kx, Uc, Ud, Ndisr, s)
        # Pickle the dipolarBEC object
        with open(f'run_k_{i}_{kx}.pkl', 'wb') as f:
            dill.dump(run_k, f)

for t in tqdm(tdar):
    visc0_kx = [[], [], []]
    visc_kx = [[], [], []]
    for kx_index, kx in enumerate(tqdm(kxar)):
        for i, s in enumerate(sigmas):
            # Unpickle the dipolarBEC object
            with open(f'run_k_{i}_{kx}.pkl', 'rb') as f:
                run_k = dill.load(f)
            visc0_kx[i].append(run_k.visc_k_0(ny, nbs[i]))    
            visc_kx[i].append(run_k.visc_k_time(ny, t, nbs[i]))
    for i in range(len(visc_t)):
        visc_t[i].append(intg(kxar, visc_kx[i])/intg(kxar,visc0_kx[2]))'''

In [1]:
plt.figure(figsize=(5,4))

for i in range(len(visc_t)):
    plt.plot(tdar, visc_t[i], label=r'$\sigma = {}$'.format(sigmas[i]))

plt.xlabel(Ust+r"$\,t$", fontsize=14)
plt.ylabel(r"$\eta/\eta_0$", fontsize=14)
plt.legend(loc='best', fontsize=14#tplt.title(r"$N = {}$ tubes, $n = {}$ tube distance, $U_d/U_c = {}$".format(Ntubes,ny, Ud), fontsize=14)
plt.savefig(os.path.join(fpath, r'visc_vs_t_3sigma.jpg'), format='jpg', bbox_inches='tight')
plt.savefig(os.path.join(fpath, r'visc_vs_t_3sigma.pdf'), format='pdf', bbox_inches='tight')

plt.show()

NameError: name 'plt' is not defined

In [ ]:
# fourier transform of viscosity vs omega for a fixed Ud/Uc, Ntubes, sigma, ny

Ntubes = N0
ny = Ntubes-1
sigma = sigmas[1]
gamma = 0.01

odar = np.arange(-20, 20, .02)
kxar = np.linspace(kxs[0], kxs[1], Nmesh)

nb = np.random.uniform(1-sigma, 1+sigma, Ntubes)
visc_or= [[], [], []]
visc_oi= [[], [], []]

for o in tqdm(odar):
    visc_kx = [[], [], []]
    for kx in tqdm(kxar):
        for i, ud in enumerate(Uds):
            run_k = dipolarBEC.dipolarBEC(Ntubes, kx, Uc, ud, Ndisr, sigma)
            visc_kx[i].append(run_k.visc_k_om(ny, o, nb, gamma))
    for i in range(len(visc_or)):
        visc_or[i].append(np.real(intg(kxar, visc_kx[i])))
        visc_oi[i].append(np.imag(intg(kxar, visc_kx[i])))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10,6))

for i in range(len(visc_or)):
    ax1.plot(odar, visc_or[i], label=Ust+Ucs+r'= {}'.format(Uds[i]))
    ax2.plot(odar, visc_oi[i], label=Ust+Ucs+r'= {}'.format(Uds[i]))

ax1.set_xlabel(r"$\omega$")
ax1.set_ylabel(r"Shear Viscosity, Re")
ax2.set_xlabel(r"$\omega$")
ax2.set_ylabel(r"Shear Viscosity, Im")
plt.legend(loc='best')
fig.tight_layout()
#fig.suptitle(r"$N = {}$ tubes, $n = {}$ tube distance, $\sigma = {}$".format(Ntubes,ny, sigma))
plt.savefig(os.path.join(fpath, r'visc_vs_om_3ud_sig={}_N={}.jpg'.format(sigma,Ntubes)), format='jpg', bbox_inches='tight')
plt.savefig(os.path.join(fpath, r'visc_vs_om_3ud_sig={}_N={}.pdf'.format(sigma,Ntubes)), format='pdf', bbox_inches='tight')


plt.show()

In [ ]:
# viscosity vs ny for a fixed Ud/Uc, Ntubes, sigma, t=1

'''Ntubes = int(Ns[0]/10)
sigma = sigmas[2]

t = t2

ndar = np.arange(1, Ntubes-1, 1)
kxar = np.linspace(kx_small, kx_large, Nmesh)

nb = np.random.uniform(1-sigma, 1+sigma, Ntubes)
visc_n = [[], [], []]

for ny in tqdm(ndar):
    visc_kx = [[], [], []]
    for kx in kxar:
        for i, ud in enumerate(Uds):
            run_k = dipolarBEC.dipolarBEC(Ntubes, kx, Uc, ud, Ndisr, sigma)
            visc_kx[i].append(run_k.visc_k_time(ny, t, nb))
    for i in range(len(visc_n)):
        visc_n[i].append(intg(kxar, visc_kx[i]))'''

In [ ]:
'''plt.figure(figsize=(5,4))

for i in range(len(visc_n)):
    plt.plot(ndar, visc_n[i], label=Ust+Ucs+r'= {}'.format(Uds[i]), marker=markers[i])

plt.xlabel(r"$n_y$", fontsize=14)
plt.ylabel(r"Shear Viscosity at fixed t", fontsize=14)
plt.legend(loc='best', fontsize=14)
#plt.title(r"$N = {}$ tubes, $\sigma = {}, t = {}$".format(Ntubes, sigma,t), fontsize=14)
plt.savefig(os.path.join(fpath, r'visc_vs_ny_t={}.jpg'.format(t)), format='jpg', bbox_inches='tight')
plt.savefig(os.path.join(fpath, r'visc_vs_ny_t={}.pdf'.format(t)), format='pdf', bbox_inches='tight')

plt.show()'''

In [ ]:
#csi vs Ntubes for sk and three sigma

#np.seterr(under='ignore')
#INtar = 1.0 / Ntar

#csi = [[] for _ in range(len(sigmas))]

#for i in range(len(sigmas)):
    #coefficients = np.polyfit(INtar, iprN_sk[i], 1)
    #csi[i].append(1/coefficients[1])

#plt.figure(figsize=(5,4))
#for i in range(len(csi)):
    #plt.plot(Ntar, csi[i], label=r'$\sigma = {}$'.format(sigmas[i]), marker=markers[i])
    #plt.xscale('log')
    #plt.yscale('log')
    #plt.ylabel(r"\xi", fontsize=14)
    #plt.xlabel(r"$N_{\mathrm{Tubes}}$", fontsize=14)
    #plt.legend(loc='best', fontsize=14)
    #save plot in jpg and pdf
    #plt.savefig(os.path.join(fpath, filename) + ".jpg", format='jpg', bbox_inches='tight')
    #plt.savefig(os.path.join(fpath, filename) + ".pdf", format='pdf', bbox_inches='tight')
    #plt.show()